## Segmentation and image contour

In [24]:
import cv2
import numpy as np

path = 'images/cap4/'

In [25]:
def show_image(img,title="img",waitKey=True):
    cv2.imshow(title,img)
    if waitKey:
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [3]:
img = cv2.imread(path+'shapes2.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#it's better to apply canny before. For example, without Canny the number of contour 
#would be 1!

canny = cv2.Canny(gray,20,200)
copy = np.copy(canny)
contours, hierarchy = cv2.findContours(copy,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
print("Contours found: ",str(len(contours)))
#with -1 we draw all the contours
cv2.drawContours(img,contours,-1,(0,255,0),3)
show_image(img)


Contours found:  5


If we print <b>contours</b> we obtain a list of list. In this case we have 5 contours, so we have 5 lists. Each element in each list is the coordinate of a point belonging to that contour. As parameters in findContours, at the approximation, we can use:
    
    cv2.CHAIN_APPROX_NONE: we store ALL the the points of a contour
    cv2.CHAIN_APPROX_SIMPLE: we store only the important points -> start and ending points

Another parameter is the **retrieval mode**:

    cv2.RETR_LIST: retrieve all the contours
    cv2.RETR_EXTERNAL: retrieve external or outer contours only
    ... other retrieval method but these are the most useful
If for example we have some holes in the shapes, with RETR_LIST we can retrieve also the contours of the holes.

### Area of shapes

To compute the area of the contours we can simply use **cv2.contourArea(contour)**
Let's calculate the areas and then sort the contours for increasing ara

In [58]:
def get_contour_areas(contours):
    areas = []
    for contour in contours:
        areas.append(cv2.contourArea(contour))
    return areas

print("Areas before sorting: ",get_contour_areas(contours))
#let's sort:
sorted_contours = sorted(contours,key=cv2.contourArea)
print("Area after sorting: ",get_contour_areas(sorted_contours))

#draw a center:
copy = np.copy(img)
shape=1
for c in sorted_contours:
    center = np.mean(c,axis=0)
    center =  ((int(center[0,0])-5,int(center[0,1])+10))
    cv2.putText(copy,str(shape),center,cv2.FONT_HERSHEY_PLAIN,1,(0,0,255),2)
    shape+=1
show_image(copy,"Shapes ordered for area")

Areas before sorting:  [8511.5, 7404.0, 12975.0, 23456.0, 8874.0]
Area after sorting:  [7404.0, 8511.5, 8874.0, 12975.0, 23456.0]


### Sorting left to right
The solutions seen so far for calculating the center of the shape is not ideal, by the moment that it depends on the regularity of the shape. For a complex and unregular shape it wont show the center. In order to get the right center we need to use the momentum.
In particularly we need to compute the center of mass of the shapes.
**cv2.Moments(c)** allows to compute the moments of the shape. In particular:

    M['m00'] correspond to the Area
    M['m10'] correspond to the moment of inertia wrt x-axis
    M['m01'] correspond to the moment of inertia wrt y-axis

In [10]:
img = cv2.imread(path+'shapes2.jpg')
copy = np.copy(img)

def label_contour_center(image,c):
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    center = (cx,cy)
    cv2.circle(copy,center,10,(0,0,255),-1)
    return copy,center

centers = []
for c in contours:
    orig,center = label_contour_center(copy,c)
    centers.append(center)
    
show_image(orig)

centers_left_to_right = np.sort(centers,axis=0)

for (i,c) in enumerate(centers_left_to_right):
    center = (c[0],c[1])
    cv2.putText(copy,str(i+1),center,cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),2)
show_image(copy)


With **cv2.boundingRect(c)** we can have the x,y of the top-left corner, width and heigth of the bounding rectangle that contains the shape

In [8]:
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(copy,(x,y),(x+w,y+h),(255,0,0),3)
    
show_image(copy)
    

## Convex hull
Convex hull is a poligon created using all the extreme contour points of a shape. It represents the tiniest poligon that can contain the shape.
Let's take the image of an hand and apply all the necessary to obtain the contour.

In [32]:
img = cv2.imread(path+'hand.png')
copy = np.copy(img)
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(img_gray,20,170)
contours,hierarchy = cv2.findContours(canny,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
cv2.drawContours(copy,contours,-1,(0,255,0),3)
show_image(copy,"contours")

#there are some internal contours that we want to discard. Let's sort using area
main_contour = sorted(contours,key=cv2.contourArea,reverse=False)[-1]
copy2 = np.copy(img)
cv2.drawContours(copy2,main_contour,-1,(0,255,0),3)
show_image(copy2,"Main contour")

hull = cv2.convexHull(main_contour)
#hull among square brakets because drawContours want a list as parameter
cv2.drawContours(copy2,[hull],-1,(0,255,0),2)
show_image(copy2,"Hull")


ValueError: too many values to unpack (expected 2)

# Shape matching
We have a shape and, in another image, we want to find all the shapes that are similar to the given one, to an arbitrary scale and/or roto-traslation

In [89]:
#I don't know why this doesn't work
shapes = cv2.imread(path+'shapes3.jpg')
target = cv2.imread(path+'target.jpg',cv2.IMREAD_GRAYSCALE)
shapes_gray = cv2.cvtColor(shapes,cv2.COLOR_BGR2GRAY)

ret, th_shapes = cv2.threshold(shapes_gray,127,255,1)
ret, th_target = cv2.threshold(target,127,255,1)

target_contours,hierarchy = cv2.findContours(th_target,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
shapes_contours,hierarchy = cv2.findContours(th_shapes,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

target_contour = sorted(target_contours,key=cv2.contourArea)[-1]


for c in shapes_contours:
    match = cv2.matchShapes(target_contour,c,1,0.0)
    if match < 0.55:
        closest_contour = c
    else:
        closest_contour = []
cv2.drawContours(shapes,closest_contour,-1,(0,255,0),3)
show_image(shapes)


## Line detection - Hough lines
![title](images/cap4/hough_lines.jpg)

In [4]:
img = cv2.imread(path+'lines.jpg')
img = cv2.resize(img,None,fx=0.5,fy=0.5)
copy = np.copy(img)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,70,200)
# rho accuracy of 1, theta accuracy of pi/180 (so 1 degree), line threshold of 240 points on a line
lines = cv2.HoughLines(edges,1,np.pi/180,240)

for line in lines:
    for rho,theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0= a*rho
        y0 = b*rho
        x1 = int(x0 + 1000 *(-b))
        y1 = int(y0 + 1000 *(a))
        x2 = int(x0 - 1000 *(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(copy,(x1,y1),(x2,y2),(255,0,0),2)
show_image(copy)

### Probabilistic Hough lines
**lines = cv2.HoughLinesP(binarizedImage,rho accuracy,theta accuracy,threshold,minimum line lenght,max line gap)**

In [5]:
copy = np.copy(img)
lines = cv2.HoughLinesP(edges,1,np.pi/180,100,5,10)

for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(copy,(x1,y1),(x2,y2),(0,0,255),3)
    
show_image(copy)
    

### Circle Detection 
**cv2.HoughCircles(image, method, dp, MinDist, param1, param2, minRadius, MaxRadius)**


In [105]:
img = cv2.imread(path+'bottle_caps.jpg')
img = cv2.resize(img,None,fx=0.5,fy=0.5)
blur = cv2.medianBlur(img,5)
gray = cv2.cvtColor(blur,cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1.3,10)



HoughCircles gives in output a list of circles composed by 3 elements: **x** and **y** of the **center** and the **radius length** of each circle

In [108]:
print("Circles found: ",len(circles[0]))
copy = np.copy(blur)
for circle in circles[0,:]:
    cv2.circle(copy,(circle[0],circle[1]),circle[2],(0,255,0),2)
    cv2.circle(copy,(circle[0],circle[1]),2,(255,0,0),3)
show_image(copy)



Circles found:  16


## Blob Detection
The step for blobs detection are:
 
    1) Create a detector
    2) Give the input image
    3) Get the Keypoints
    4) Draw the keypoints

In [64]:
img = cv2.imread(path+'sunflowers.jpg',cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img,None,fx=0.5,fy=0.5)
img = img.astype(np.uint8)

detector = cv2.SimpleBlobDetector_create() 

#detect blobs:
keypoints = detector.detect(img)
blank = np.zeros((1,1))
blobs = cv2.drawKeypoints(img,keypoints,blank,(0,255,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
show_image(blobs,"Blobs")

As we can see there is nothing we can do to modify the numbers of the blobs found. That's because we are using the default consctructor for the simple blob detector. 

## MiniProject - counting circles and ellipses

https://www.learnopencv.com/blob-detection-using-opencv-python-c/

This link shows a lot of params we can exploit to obtain a perfect result.

In [94]:
img = cv2.imread(path+'blobs.jpg')          
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
h,w = img.shape[:2]


#Set filtering parameters
params = cv2.SimpleBlobDetector_Params()
params.filterByArea = False
params.filterByCircularity = False
params.filterByInertia = False
detector = cv2.SimpleBlobDetector_create(params)
keyPoints = detector.detect(gray)
blank = np.zeros((1,1))
copy = np.copy(img)
blobs = cv2.drawKeypoints(copy,keyPoints,blank,(0,0,255),cv2.DRAW_MATCHES_FLAGS_DEFAULT)
text = "Blobs found: " + str(len(keyPoints))
cv2.putText(blobs,text,(10,h-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
show_image(blobs,"All Blobs")


params = cv2.SimpleBlobDetector_Params()
params.filterByArea = False
params.filterByCircularity = True
params.minCircularity = 0.8
params.filterByInertia = True
params.minInertiaRatio = 0.6
detector = cv2.SimpleBlobDetector_create(params)
keyPoints = detector.detect(gray)
copy = np.copy(img)
blobs = cv2.drawKeypoints(copy,keyPoints,blank,(0,0,255),cv2.DRAW_MATCHES_FLAGS_DEFAULT)
text = "Circles found: " + str(len(keyPoints))
cv2.putText(blobs,text,(10,h-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
show_image(blobs,"Circles")


In [41]:
import numpy
import cv2


def gaussianBlur(img):
    sigma = 1
    k = 3*sigma
    k_size = np.int(np.ceil((2*k+1)**2))
    gk = cv2.getGaussianKernel(k_size,sigma)
    blurred = cv2.filter2D(img,-1,gk)
    blurred = cv2.filter2D(blurred,-1,gk.T)
    return blurred

def compute_convex_hull(frame):
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = gaussianBlur(gray)
    canny = cv2.Canny(gray,70,160)
    _,contours,hierarchy = cv2.findContours(canny,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    main_contour = sorted(contours,key=cv2.contourArea,reverse=False)[-1]
    hull = cv2.convexHull(main_contour)
    cv2.drawContours(frame,[hull],-1,(0,255,0),2)
    return frame

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    hull = compute_convex_hull(frame)
    cv2.imshow("hull",hull)
    if cv2.waitKey(3) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()